In [ ]:
# Assumptions 
# - 2x farm where one asset is stablecoin, e.g.  ETH/USDC
# - there is always borrow availability
# - borrow rate, trading fees, rewards are constant

In [ ]:
from math import exp, sqrt
from statistics import mean
import pandas as pd 

In [142]:
class Agent():
    
    def __init__(self, equity, price, borrow_rate, farming_rate):
        print("init")
        
        self.borrow_rate = borrow_rate
        self.farming_rate = farming_rate
        
        # Make an intial investment - 50% stablecoin, 50% borrowed token
        self.equity = equity
        self.assetA_count = equity / price  # token
        self.assetB_count = equity          # stablecoin
        self.debt_value =  self.assetA_count * price
        
        self.last_rebalance_step = 0
        self.last_rebalance_price = price
        
        self.calculate_position(0, price)
        
    def rebalance(self, step, new_price):
        print("rebalance")
        
        # Debt is already subtracted from position_value to get equity
        # Re-invest equity with a 50/50 split
        self.assetA_count = self.equity / new_price  # token
        self.assetB_count = self.equity              # stablecoin
        self.debt_value =  self.assetA_count * new_price
        
        self.last_rebalance_step = step
        self.last_rebalance_price = new_price
    
    # Calculates since last rebalance, not path-dependent
    def calculate_position(self, step, new_price):
        print("calculate_pnl")
        steps_since_rebalance = step - self.last_rebalance_step

        price_change_ratio = new_price / self.last_rebalance_price
        lp_multiplier = sqrt(price_change_ratio)
        
        # From https://docs.google.com/spreadsheets/d/1mGZDKgtRNXilSQT4U5YbqNPCSqMiPayuqvD8Xx4Keqo/
        position_value = (2*self.assetB_count*lp_multiplier) + 2*mean([self.assetB_count*lp_multiplier, self.assetB_count])*(exp(self.farming_rate*steps_since_rebalance)-1)
        
        debt_value = self.assetA_count * new_price * exp(self.borrow_rate*steps_since_rebalance)
        equity = position_value - debt_value
        
        self.position_value = position_value
        self.debt_value = debt_value
        self.equity = equity

#        print ("steps_since_rebalance=" + str(steps_since_rebalance))
#        print ("price_change_ratio=" + str(price_change_ratio))
#        print ("lp_multiplier=" + str(lp_multiplier))
#        print ("position_value=" + str(self.position_value))
#        print ("debt_value=" + str(self.debt_value))
#        print ("equity_value=" + str(self.equity))

    def action(self, step, new_price):
        print ("\nACTION...  price is " + str(new_price))
        print ("self.last_rebalance_price=" + str(self.last_rebalance_price))
        
        self.calculate_position(step, new_price)

        # Naive rebalancer - trigger when price moves ~10% in either direction
        if ((new_price >= self.last_rebalance_price*1.1) or (new_price <= self.last_rebalance_price*0.9)):
            self.rebalance(step, new_price)
            self.calculate_position(step, new_price)
        


In [186]:
########## PRICE DATASET #############

# Test dataset
test_prices = [13.70, 13.70, 14.385, 15.07, 15.76, 11.65, 12.33]

# ETH dataset
eth_prices = pd.read_csv('data/Binance_ETHUSDT_1h.csv')
eth_prices['date'] = eth_prices['date'].str.slice(0,10)  # we remove time bc panadas won't parse otherwise
eth_prices['date'] = pd.to_datetime(eth_prices['date'])

# Filter on date
eth_prices = eth_prices[(eth_prices['date'] > '2022-01-01') & (eth_prices['date'] < '2022-02-01')]

# Reverse so that earliest price is first
eth_prices = eth_prices.iloc[::-1]

# Choose which dataset
# Simulation expects a dataframe with a field called 'close' 
prices = eth_prices

print (str(len(prices)) + " rows")
prices.head()



720 rows
3761.58


In [187]:
########## RUN SIMULATION ############

results = pd.DataFrame(columns = ['step', 'price', 'assetA_count', 'assetB_count', 'position_value', 'debt_value', 'equity', 'last_rebalance_step'])

INITIAL_EQUITY = 10000
BORROW_APR = 85/100        # token borrow rate (%)
FARMING_APR = 100/100      # farming and all rewards rate (%)
STEPS_PER_YEAR = 365*24    # hourly data

# Assume a time step is 1 day
agent = Agent(
    INITIAL_EQUITY,        # initial equity
    prices['close'].iloc[0],         # initial token_price 
    BORROW_APR/365,    # borrow rate per time step 
    FARMING_APR/365    # farming rate at 1x leverage per time step
)

# Drop first row
prices = prices.iloc[1: , :]

step = 0
for row in prices.rows:
    step += 1

    agent.action(step, row['close']) 
    
    results = df.append ({
        'step': step,
        'price': p,
        'assetA_count' : agent.assetA_count,
        'assetB_count' : agent.assetB_count,
        'position_value': agent.position_value,
        'debt_value': agent.debt_value,
        'equity': agent.equity,
        'last_rebalance_step': agent.last_rebalance_step
        },
        ignore_index=True)

results.head()



#TODO
# put in real price data 
# brownie tests!  this is fragile!
# summary stats
# try rebalance strats
# deal with nuances - e.g. francium autocompounds rewards every 15m, compounding of borrow?  https://docs.francium.io/product/protocol-parameters
# extend to 3x and other leverages

# entry_fee = 0
# exit_fee = 0
# controller_fee = 0.001  
# reinvestent_fee = 0.039 
# tx_fee = 0  # network fee
# performance_fee = 0


init
calculate_pnl


AttributeError: 'DataFrame' object has no attribute 'rows'